In [1]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
file_path = "../data/data"

In [3]:
# we are only training for the first page, so we will remove all the other pages
def filterFirstPage():
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if not file.endswith('00.fmt') and not file.endswith('00.png') and not file.endswith('00.xml'):
                os.remove(os.path.join(root, file))

In [4]:
# check if all the train images are the same size, and how many images we have in total

def getImageSizes(file_path):
    sizes = set()
    count = 0
    
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if file.endswith('.png'):
                im = cv2.imread(os.path.join(root, file))
                sizes.add(im.shape)
                count += 1
    print(sizes)
    print(count)

In [5]:
getImageSizes(file_path)

{(3300, 2560, 3)}
900


In [6]:
dataset = []

def checkCoordinates(file_path):
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if file.endswith('.xml'):
                tree = ET.parse(os.path.join(root, file))
                data = createImageDict(tree.getroot())
                dataset.append(data)

In [7]:
def createImageDict(root):
    # calculate the percentage from top left right bottom for each box:page
    pageX_start = 0
    pageX_end = int(root[4][0].text)
    pageY_start = 0
    pageY_end = int(root[4][1].text)
    
    width = int(root[4][0].text)
    height = int(root[4][1].text)
    
    boxX_start = int(root[6][4][0].text)
    boxY_start = int(root[6][4][1].text)
    boxX_end = int(root[6][4][2].text)
    boxY_end = int(root[6][4][3].text)
    
    
    left = (boxX_start - pageX_start)/width
    top = (boxY_start - pageY_start)/height
    right = (pageX_end - boxX_end)/width
    bottom = (pageY_end - boxY_end)/height
    
    
    return {
        "left": left,
        "right": right,
        "top": top,
        "bottom": bottom
    }
    

In [8]:
# check the range of each field in the dataset

def checkRange():
    num_fields = len(dataset[0])
    data_array = np.array([[d[field] for field in dataset[0]] for d in dataset])

    # Calculate the minimum and maximum values for each field
    min_values = np.min(data_array, axis=0)
    max_values = np.max(data_array, axis=0)

    # Print the ranges for each field
    for i, field in enumerate(dataset[0]):
        print(f"{field} range: {min_values[i]} to {max_values[i]}")


In [9]:
checkCoordinates(file_path)
print(len(dataset))
checkRange()
# Missing 580

899
left range: 0.138671875 to 0.23671875
right range: 0.19375 to 0.283984375
top range: 0.04939393939393939 to 0.11
bottom range: 0.7866666666666666 to 0.8366666666666667
